# Results Explorer for lorenz models

includes plotting the sindy coefficients $\Xi$, loss metrics and trajectories for different initial conditions 

### Choose Model

In [1]:
#CHOOSE MODEL
model_path = "checkpoints/kathleenReplica_1"
#CHOOSE MODEL

In [4]:
import sys
sys.path.append('../')
from trainer import SINDy_trainer
import jax.numpy as jnp¨

#exmp input needs to have 128 as second dimension
exmp_input = jnp.ones(1, 128)
trainer = SINDy_trainer.load_from_checkpoint(model_path, exmp_input)

xi_raw = trainer.state.params['sindy_coefficient']
mask = trainer.mask

xi = xi_raw * mask

print(xi)

TypeError: SINDy_trainer.load_from_checkpoint() missing 1 required positional argument: 'exmp_input'